# Lab 8: Local search algorithms

In [19]:
import numpy as np
import random

## Hill climbing

Hill climbing is a mathematical optimization technique which belongs to the family of local search. It is an iterative algorithm that starts with an arbitrary solution to a problem, then attempts to find a better solution by incrementally changing a single element of the solution. If the change produces a better solution, an incremental change is made to the new solution, and so on until no further improvements can be found.

One of the different part between hill climbing and gradient descent is that hill climbing does not require the gradient of the problem being optimized. This is a very great feature because it allows hill climbing to be used on problems that are not differentiable or have an unknown gradient, which means it is perfect for discrete optimization problems.

Here is an example of a simple hill climbing algorithm:


In [6]:
def objective(x):
    return - (x - 3)**2 + 10

def hill_climbing(start_x, step_size=0.1, max_iterations=500):
    x = start_x
    for _ in range(max_iterations):
        candidate_x = x + np.random.choice([-step_size, step_size])
        if objective(candidate_x) > objective(x):  
            x = candidate_x
    return x, objective(x)

best_x, best_value = hill_climbing(start_x=np.random.uniform(0, 6))
print(f"Hill Climbing: Best x = {best_x:.4f}, Best Value = {best_value:.4f}")

Hill Climbing: Best x = 3.0174, Best Value = 9.9997


The n-queens problem is a classic problem in computer science. The problem is to place n queens on an n×n chessboard so that no two queens threaten each other. Thus, a solution requires that no two queens share the same row, column, or diagonal.

This is an example solution of 4-queens problem:

![4-queens](https://media.geeksforgeeks.org/wp-content/uploads/20230814111654/Solution-Of-4-Queen-Problem.png)

In the `fitness` function, we count the number of non-attacking pairs of queens. The optimal solution is when the fitness score is equal to `n * (n - 1) // 2`, which means no two queens threaten each other. The return value of the `fitness` function is higher when the solution is better. So the goal is to maximize the fitness score.

In the `generate_neighbor` function, we generate a new board by moving one queen to a new row. The `hill_climbing` function is the main algorithm that solves the n-queens problem using hill climbing. It starts with a random initial state and generates a new board by moving one queen to a new row. If the new board has a better fitness score, the algorithm accepts the move. The algorithm stops when it reaches the optimal solution or the maximum number of iterations.


In [17]:
def fitness(board):
    """ Counts the number of non-attacking pairs of queens """
    n = len(board)
    max_pairs = n * (n - 1) // 2  # Total possible non-attacking pairs
    attacking_pairs = 0

    for i in range(n):
        for j in range(i + 1, n):
            if abs(board[i] - board[j]) == abs(i - j):  # Diagonal attack
                attacking_pairs += 1

    return max_pairs - attacking_pairs  # Higher is better

def generate_neighbor(board, rng):
    """ Generate a new board by swapping two queen positions (ensuring unique rows) """
    n = len(board)
    new_board = board[:]
    i, j = rng.choice(n, size=2, replace=False)  # Pick two different columns
    new_board[i], new_board[j] = new_board[j], new_board[i]  # Swap their rows
    return new_board

def hill_climbing(n=8, max_iterations=1000, seed=42):
    """ Solves the N-Queens problem using hill climbing with unique row constraints """
    rng = np.random.default_rng(seed)  # Fixed seed for reproducibility
    board = rng.permutation(n)  # Start with a permutation (ensures unique row positions)
    best_fitness = fitness(board)

    for _ in range(max_iterations):
        new_board = generate_neighbor(board, rng)
        new_fitness = fitness(new_board)
        if new_fitness > best_fitness:
            board, best_fitness = new_board, new_fitness
        if best_fitness == (n * (n - 1) // 2):
            break

    return board, best_fitness

solution, best_fit = hill_climbing(n=8, seed=42)
print("Final Board State (Row Positions of Queens):", solution)
print("Fitness Score:", best_fit, "(Higher is better)")

Final Board State (Row Positions of Queens): [5 2 6 3 0 7 1 4]
Fitness Score: 28 (Higher is better)


In [18]:
def print_chessboard(board):
    n = len(board)
    for row in range(n):
        for col in range(n):
            if board[col] == row:
                print("Q", end=" ")
            else:
                print(".", end=" ")
        print()
        
print_chessboard(solution)

. . . . Q . . . 
. . . . . . Q . 
. Q . . . . . . 
. . . Q . . . . 
. . . . . . . Q 
Q . . . . . . . 
. . Q . . . . . 
. . . . . Q . . 


## K-Median Clustering

K-Median Clustering is a clustering algorithm that aims to partition n data points into k clusters. The goal is to minimize the sum of distances between each data point and the median of its cluster. The median is the data point that minimizes the sum of distances to all other data points in the cluster.

$$
\text{Cost} = \sum_{i=1}^{n} \min_{j=1}^{k} \text{dist}(x_i, c_j)
$$

where $x_i$ is the data point, $c_j$ is the median of cluster $j$, and $\text{dist}(x_i, c_j)$ is the distance between $x_i$ and $c_j$. The cost function is the sum of distances between each data point and the median of its cluster. The goal is to minimize this cost function.

The `distance` function computes the Manhattan distance between two points. The `total_cost` function computes the total sum of distances from each point to its nearest median. The `generate_neighbor` function swaps one of the medians with a random non-median data point. The `k_median_local_search` function is the main algorithm that solves the k-median clustering problem using local search. It starts with k random medians and generates a new neighbor by swapping one of the medians with a random non-median data point. The algorithm stops when it reaches the optimal solution or the maximum number of iterations.

In [23]:
data_points = np.array([
    [2, 3], [5, 4], [3, 6], [8, 8], [1, 1],
    [7, 5], [9, 9], [6, 2], [4, 7], [10, 6]
])

def distance(p1, p2):
    """ Computes Manhattan distance between two points """
    return np.sum(np.abs(np.array(p1) - np.array(p2)))

def total_cost(medians, data):
    """ Computes total sum of distances from each point to its nearest median """
    return sum(min(distance(point, median) for median in medians) for point in data)

def generate_neighbor(medians, data):
    """ Swap one of the medians with a random non-median data point """
    new_medians = medians.copy()
    swap_out = random.choice(medians)  # Pick a median to swap out
    swap_in = random.choice([p for p in data if p.tolist() not in medians])  # Pick a new median
    new_medians.remove(swap_out)
    new_medians.append(tuple(swap_in))
    return new_medians

def k_median_local_search(data, k, max_iterations=1000, seed=42):
    """ Local search for k-median clustering with proper integer conversion """
    random.seed(seed)
    np.random.seed(seed)
    
    # Start with k random medians
    initial_medians = [tuple(map(int, p)) for p in random.sample(data.tolist(), k)]
    best_medians = initial_medians
    best_cost = total_cost(best_medians, data)
    
    for _ in range(max_iterations):
        new_medians = generate_neighbor(best_medians, data)
        new_medians = [tuple(map(int, m)) for m in new_medians]
        new_cost = total_cost(new_medians, data)
        
        if new_cost < best_cost:
            best_medians, best_cost = new_medians, new_cost
    
    return best_medians, best_cost

optimal_medians, min_cost = k_median_local_search(data_points, k=3)
print("Optimal Medians:", optimal_medians)
print("Minimum Cost:", min_cost)

Optimal Medians: [(1, 1), (8, 8), (5, 4)]
Minimum Cost: 23


## Genetic Algorithm

Genetic Algorithm is a metaheuristic optimization algorithm inspired by the process of natural selection. It is based on the principles of genetics and natural selection. The algorithm starts with a random population of candidate solutions, then evolves the population over multiple generations. Each generation consists of selection, crossover, and mutation steps.

1. Initialize Population: Start with a set of random k-medians.
2. Evaluate Fitness: Compute the total clustering cost for each candidate.
3. Selection: Pick better solutions (medians with lower cost).
4. Crossover (Recombination): Combine solutions to create offspring.
5. Mutation: Slightly change some solutions to maintain diversity.
6. Repeat until convergence or a maximum number of generations.

Here is an example implementation of the genetic algorithm for k-median clustering:

In [24]:
def initialize_population(data, k, pop_size):
    """Creates an initial population of k-median solutions."""
    return [random.sample([tuple(p) for p in data.tolist()], k) for _ in range(pop_size)]

def select_parents(population, data):
    """Selects the best individuals (lower cost) to be parents."""
    return sorted(population, key=lambda m: total_cost(m, data))[:len(population)//2]

def crossover(parent1, parent2):
    """Performs crossover by mixing medians from two parents."""
    crossover_point = len(parent1) // 2
    child = parent1[:crossover_point] + parent2[crossover_point:]
    return list(set(child))[:len(parent1)]  # Ensure k elements and no duplicates

def mutate(solution, data, mutation_rate=0.1):
    """Randomly mutates a median by replacing one element."""
    if random.random() < mutation_rate:
        replace_index = random.randint(0, len(solution) - 1)
        new_median = tuple(random.choice(data.tolist()))
        solution[replace_index] = new_median
    return solution

def genetic_algorithm_k_median(data, k, pop_size=10, generations=100, mutation_rate=0.1):
    """Genetic Algorithm for k-Median Optimization"""
    random.seed(42)
    np.random.seed(42)

    # Initialize population
    population = initialize_population(data, k, pop_size)

    for _ in range(generations):
        # Select best solutions as parents
        parents = select_parents(population, data)

        # Create next generation via crossover and mutation
        offspring = []
        while len(offspring) < pop_size:
            p1, p2 = random.sample(parents, 2)
            child = crossover(p1, p2)
            mutated_child = mutate(child, data, mutation_rate)
            offspring.append(mutated_child)

        population = offspring  # Replace old generation

    best_solution = min(population, key=lambda m: total_cost(m, data))
    return best_solution, total_cost(best_solution, data)

optimal_medians, min_cost = genetic_algorithm_k_median(data_points, k=3)
print("Optimal Medians:", optimal_medians)
print("Minimum Cost:", min_cost)

Optimal Medians: [(2, 3), (5, 4), (9, 9)]
Minimum Cost: 23
